# 1. Setup

Import librarys and load database into juypiter

### Connect to data base

In [76]:
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geopandas as gpd
from datetime import datetime
from sqlalchemy import text
from sqlalchemy import inspect
from sqlalchemy import create_engine
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="sydney"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn
    
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

srid = 4326

In [77]:
db, conn = pgconnect(credentials)

Connected successfully.


In [78]:
sql_create_schema = """
DROP SCHEMA IF EXISTS sydney CASCADE;
CREATE SCHEMA IF NOT EXISTS sydney;
"""
query(conn, sql_create_schema)
inspect(db).get_schema_names()


Error encountered: 
This result object does not return rows. It has been closed automatically.


['carhire', 'information_schema', 'nswfuel', 'prices', 'public', 'sydney']

### Import data into python and data cleaning

In [79]:
SA2_Regions = gpd.read_file('SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2_Regions = SA2_Regions[SA2_Regions['GCC_CODE21'] == "1GSYD"]

def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

SA2_Regionsog = SA2_Regions.copy() 
SA2_Regions['geom'] = SA2_Regions['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
SA2_Regions = SA2_Regions.drop(columns="geometry")
SA2_Regions


SA2_Regions

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geom
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.413733024921 -33.465580583...
29,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37484081570685 -33.5005199...
30,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.20449037540152 -33.5328022...
31,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37193611462118 -33.4369790...
32,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.32348639265098 -33.4277852...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.07362997413264 -34.0563789...
638,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.05006441218998 -34.0215774...
639,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((150.99568346574816 -34.0536082...
640,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.03954821100714 -34.0417452...


Businesses dataset:

change the name of columns start with number, to match the SQL formate\n

set sa2_code as primary key, check if any row has empty primary key

In [80]:
businesses = pd.read_csv('Businesses.csv')

column_mapping = {
    '0_to_50k_businesses': 'businesses_0-50k',
    '50k_to_200k_businesses': 'businesses_50k-200k',
    '200k_to_2m_businesses': 'businesses_200k-2m',
    '2m_to_5m_businesses': 'businesses_2m-5m',
    '5m_to_10m_businesses': 'businesses_5m-10m',
    '10m_or_more_businesses': 'businesses_10m_or_more',
    'total_businesses': 'total_businesses'
}
businesses.rename(columns=column_mapping, inplace=True)

# Check for missing values in the 'sa2_code' column, as this is the primary key
missing_sa2_code = businesses['sa2_code'].isnull()
print (f"{businesses['sa2_code'].isnull().sum()} rows with empty primary key have been deleted")
businesses = businesses[~missing_sa2_code]

businesses.head()

0 rows with empty primary key have been deleted


,industry_code,industry_name,sa2_code,sa2_name,businesses_0-50k,businesses_50k-200k,businesses_200k-2m,businesses_2m-5m,businesses_5m-10m,businesses_10m_or_more,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16


In [81]:
businesses.dtypes

industry_code             object
industry_name             object
sa2_code                   int64
sa2_name                  object
businesses_0-50k           int64
businesses_50k-200k        int64
businesses_200k-2m         int64
businesses_2m-5m           int64
businesses_5m-10m          int64
businesses_10m_or_more     int64
total_businesses           int64
dtype: object

Income dataset:

In [82]:
income = pd.read_csv('Income.csv')

missing_sa2_code = income['sa2_code21'].isnull()
print (f"{income['sa2_code21'].isnull().sum()} rows with empty primary key have been deleted")
income = income[~missing_sa2_code]

income.head()


0 rows with empty primary key have been deleted


,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981


Population dataset:

In [83]:
stops = pd.read_csv('Stops.csv')

# Check for missing values in the 'sa2_code' column, as this is the primary key
missing_stop_id = stops['stop_id'].isnull()
print (f"{stops['stop_id'].isnull().sum()} rows with empty primary key have been deleted")
stops = stops[~missing_stop_id]

stops.head()

0 rows with empty primary key have been deleted


,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code
0,200039,200039.0,"Central Station, Eddy Av, Stand A",-33.882206,151.206665,NaN,200060,0,NaN
1,200054,200054.0,"Central Station, Eddy Av, Stand D",-33.882042,151.206991,NaN,200060,0,NaN
2,200060,NaN,Central Station,-33.884084,151.206292,1.0,NaN,0,NaN
3,201510,NaN,Redfern Station,-33.891690,151.198866,1.0,NaN,0,NaN
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",-33.893329,151.198882,NaN,201510,0,NaN


In [84]:
schools_future = gpd.read_file('catchments/catchments_future.shp')

schools_futureog = schools_future.copy() 
schools_future['geom'] = schools_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
schools_future = schools_future.drop(columns="geometry")
schools_future['ADD_DATE'] = schools_future['ADD_DATE'].apply(lambda x: '0000-00-00' if x is None else datetime.strptime(str(x), "%Y%m%d"))
schools_future

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geom
0,8416,HIGH_COED,Ku-ring-gai HS,2023-01-14,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.19848917708944 -33.5398987...
1,8161,HIGH_BOYS,Randwick BHS,2020-02-20,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.27151530428182 -33.9140183...
2,8539,HIGH_COED,SSC Blackwattle Bay,2022-06-09,0,0,0,0,0,0,0,0,0,0,0,2024,2024,MULTIPOLYGON (((151.15292370935092 -33.8393921...
3,8400,HIGH_COED,St Ives HS,2023-01-14,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.17793729938725 -33.6982001...
4,8555,HIGH_COED,Rose Bay SC,2020-02-20,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.28072275958445 -33.8328728...
5,8556,CENTRAL_HIGH,Alexandria Park CS,2020-02-20,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.1949653506184 -33.88876468...
6,8913,HIGH_COED,Inner Sydney HS,2020-02-20,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2025,MULTIPOLYGON (((151.2098245099502 -33.85422949...
7,8286,HIGH_COED,Mt Annan HS,2022-03-01,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((150.77298245154256 -34.0251624...
8,8584,HIGH_COED,Elizabeth Macarthur HS,2022-03-01,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((150.7323512413543 -34.01465804...
9,8290,HIGH_COED,John Edmondson HS,2019-05-20,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((150.70497435250746 -33.9042617...


In [85]:
schools_primary = gpd.read_file('catchments/catchments_primary.shp')

schools_primaryog = schools_primary.copy() 
schools_primary['geom'] = schools_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
schools_primary = schools_primary.drop(columns="geometry")
schools_primary['ADD_DATE'] = schools_primary['ADD_DATE'].apply(lambda x: '0000-00-00' if x is None else datetime.strptime(str(x), "%Y%m%d"))

schools_primary.dtypes

USE_ID        object
CATCH_TYPE    object
USE_DESC      object
ADD_DATE      object
KINDERGART    object
YEAR1         object
YEAR2         object
YEAR3         object
YEAR4         object
YEAR5         object
YEAR6         object
YEAR7         object
YEAR8         object
YEAR9         object
YEAR10        object
YEAR11        object
YEAR12        object
PRIORITY      object
geom          object
dtype: object

In [86]:
schools_secondary = gpd.read_file('catchments/catchments_secondary.shp')

schools_secondaryog = schools_secondary.copy() 
schools_secondary['geom'] = schools_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
schools_secondary = schools_secondary.drop(columns="geometry")
schools_secondary['ADD_DATE'] = schools_secondary['ADD_DATE'].apply(lambda x: '0000-00-00' if x is None else datetime.strptime(str(x), "%Y%m%d"))


schools_secondary.dtypes

USE_ID        object
CATCH_TYPE    object
USE_DESC      object
ADD_DATE      object
KINDERGART    object
YEAR1         object
YEAR2         object
YEAR3         object
YEAR4         object
YEAR5         object
YEAR6         object
YEAR7         object
YEAR8         object
YEAR9         object
YEAR10        object
YEAR11        object
YEAR12        object
PRIORITY      object
geom          object
dtype: object

In [87]:
polling = pd.read_csv('PollingPlaces2019.csv')

# Check for missing values in the 'FID' column
missing_FID = polling['FID'].isnull()
print(f"{missing_FID.sum()} rows with empty primary key have been deleted")
polling = polling[~missing_FID]

# Check for missing values in the 'the_geom' column
missing_geom = polling['the_geom'].isnull()
print(f"{missing_geom.sum()} rows with empty geom have been deleted")
polling = polling[~missing_geom]

# Convert the string representation of point geometries to Shapely Point objects
polling['the_geom'] = polling['the_geom'].apply(lambda x: Point(float(x.split()[1].strip("(")), float(x.split()[2].strip(")"))))

# Convert the Shapely Point objects to WKTElement objects
polling['the_geom'] = polling['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

# Drop latitude and longitude columns
polling = polling.drop(columns=['latitude', 'longitude'])

# Convert premises_post_code to integer
polling['premises_post_code'] = polling['premises_post_code'].astype(int)

polling.dtypes

0 rows with empty primary key have been deleted
140 rows with empty geom have been deleted


FID                            object
state                          object
division_id                     int64
division_name                  object
polling_place_id                int64
polling_place_type_id           int64
polling_place_name             object
premises_name                  object
premises_address_1             object
premises_address_2             object
premises_address_3             object
premises_suburb                object
premises_state_abbreviation    object
premises_post_code              int64
the_geom                       object
dtype: object

### load dataset into SQL data base

In [88]:
businesses.to_sql("businesses", con=conn, schema="sydney", if_exists='replace', index=False)
income.to_sql("income", con=conn, schema="sydney", if_exists='replace', index=False)
population.to_sql("population", con=conn, schema="sydney", if_exists='replace', index=False)
stops.to_sql("stops", con=conn, schema="sydney", if_exists='replace', index=False)


718

In [89]:

SA2_Regions.to_sql("sa2_regions", con=conn, schema="sydney", if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_future.to_sql("schools_future", con=conn, schema="sydney", if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_primary.to_sql("schools_primary", con=conn, schema="sydney", if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_secondary.to_sql("schools_secondary", con=conn, schema="sydney", if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
polling.to_sql('polling', con=conn, schema="sydney", if_exists='replace', index=False, dtype={'the_geom': Geometry('POINT', srid)})


790

In [90]:
sql = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'sydney';
"""
query(conn, sql)

,table_name
0,businesses
1,income
2,population
3,stops
4,sa2_regions
5,schools_future
6,schools_primary
7,schools_secondary
8,polling


In [91]:
sql = """
SELECT *  from sydney.sa2_regions
"""
query(conn, sql)

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geom
0,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000200000001030000000100000085...
3,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000007E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000046...
369,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000089...
370,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000008E...
371,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E610000001000000010300000001000000A1...
